In [1]:
import pandas as pd
from obd import bts_parquet_path, bts_dat_path, bts_all_path, bts_csv_path, bts_item_context_path, train_model, preprocess, preprocess_item_context, downcast, y_name, cost_name, prob_name
from cb import df_to_adf_dat, df_to_vw_format, df_to_vw_adf_format
from utils import iter_partitions, iter_parquets
from os.path import join
from sklearn.preprocessing import OrdinalEncoder
from vowpalwabbit import Workspace
from pandas.api.types import is_integer_dtype
import numpy as np
from sklearn.linear_model import LogisticRegression
# from obp.ope import OffPolicyEvaluation, RegressionModel

In [2]:
# df = pd.read_parquet(bts_parquet_path)
# df, x_encoder = preprocess(df)
# df = downcast(df)
# df.to_parquet("processed.parquet")
# df.head()

In [27]:
df = pd.read_parquet("processed.parquet").iloc[11_000_000:]
df[cost_name] = 1 - df[cost_name]

In [28]:
df.head()

,position,user_feature_0,user_feature_1,user_feature_2,user_feature_3,user-item_affinity_0,user-item_affinity_1,user-item_affinity_2,user-item_affinity_3,user-item_affinity_4,...,user-item_affinity_75,user-item_affinity_76,user-item_affinity_77,user-item_affinity_78,user-item_affinity_79,hour,day,item_id,click,propensity_score
11000000,2,1,5,9,6,0,0,0,0,0,...,0,0,0,0,0,4,6,18,1,0.100935
11000001,1,1,5,9,6,0,0,0,0,0,...,0,0,0,0,0,4,6,7,1,0.149415
11000002,0,2,4,0,9,0,0,0,0,0,...,0,0,0,0,0,4,6,41,1,0.041715
11000003,1,2,4,0,9,0,0,0,0,0,...,0,0,0,0,0,4,6,7,1,0.149415
11000004,2,2,4,0,9,0,0,0,0,0,...,0,0,0,0,0,4,6,61,0,0.025530


In [29]:
action_context = pd.read_csv(bts_item_context_path)
action_context = action_context.drop(["Unnamed: 0"], axis=1)
action_context, ac_encoder = preprocess_item_context(action_context)
action_context.head()

,item_id,item_feature_0,item_feature_1,item_feature_2,item_feature_3
0,0,13.0,5.0,2.0,3.0
1,1,11.0,7.0,2.0,3.0
2,2,31.0,7.0,8.0,5.0
3,3,12.0,11.0,15.0,2.0
4,4,36.0,1.0,20.0,5.0


In [38]:
model = Workspace("--cb_adf --cb_type dr", enable_logging=True)
train_model(model, df, action_context)

  0%|          | 3572/1357200 [00:05<32:36, 691.95it/s]


KeyboardInterrupt: 

In [39]:
model.finish()
model.get_log()

In [37]:
model.save("obd_mtr.vw")
model.finish() 
model.get_log()

['using no cache',
 'Reading datafile = none',
 'num sources = 0',
 'Num weight bits = 18',
 'learning rate = 0.5',
 'initial_t = 0',
 'power_t = 0.5',
 'cb_type = mtr',
 'Enabled learners: gd, scorer-identity, csoaa_ldf-rank, cb_adf, shared_feature_merger',
 'Input label = CB',
 'Output pred = ACTION_SCORES',
 'average  since         example        example        current        current  current',
 'loss     last          counter         weight          label        predict features',
 '0.000000 0.000000            1            1.0       18:1:0.1            0:0     7440',
 '0.000000 0.000000            2            2.0       7:1:0.15         0:0.76     7440',
 '0.000000 0.000000            4            4.0       7:1:0.15         6:0.97     7440',
 '14.77541 29.55083            8            8.0      51:1:0.11         6:0.97     7440',
 '7.387708 0.000000           16           16.0       18:1:0.1         6:0.93     7440',
 '3.693854 0.000000           32           32.0      18:1:0.09   

In [ ]:
0.941258

In [24]:
for stat in model.get_log():
    if stat.startswith("average loss"):
        model_loss = float(stat.split(" ")[3])
bts_loss = sum(df[cost_name])/len(df)
model_loss, bts_loss

(-0.036606, 0.005501031535514294)

In [ ]:
0.00177